In [ ]:
import muda
import glob
import os
import tqdm

folder = "fold2"
jams_files_path = f"data/UrbanSound8K/augmented_data/train/{folder}"
original_jams_file_path = f"{jams_files_path}/original/jams"

deformers_with_name = []

# Setup background deformer
background_noise_deformation_name = "bgnoise"
background_audio_files_glob = "data/UrbanSound8K/background_noise/*.wav"
audio_files_glob = f"data/UrbanSound8K/audio/{folder}/*.wav"
background_deformer = muda.deformers.BackgroundNoise(files=glob.glob(background_audio_files_glob))

deformers_with_name.append((background_noise_deformation_name, background_deformer))

# Setup time stretching deformer
time_stretch_deformation_name = "stretch"
time_stretch_deformer = muda.Union(steps=[(f"time-stretch-{i}", muda.deformers.TimeStretch(rate=stretch_factor)) for i, stretch_factor in enumerate([0.81, 0.93, 1.07, 1.23])])

deformers_with_name.append((time_stretch_deformation_name, time_stretch_deformer))

# Setup pitch shifting 1
pitch_shift_1_name = "pitch1"
pitch_shift_1_deformer = muda.Union(steps=[(f"pitch-shift-1-{i}", muda.deformers.PitchShift(n_semitones=semitone_shift_factor)) for i, semitone_shift_factor in enumerate([-2, -1, 1, 2])])

deformers_with_name.append((pitch_shift_1_name, pitch_shift_1_deformer))

# Setup pitch shifting 2
pitch_shift_2_name = "pitch2"
pitch_shift_2_deformer = muda.Union(steps=[(f"pitch-shift-2-{i}", muda.deformers.PitchShift(n_semitones=semitone_shift_factor)) for i, semitone_shift_factor in enumerate([-3.5, -2.5, 2.5, 3.5])])

deformers_with_name.append((pitch_shift_2_name, pitch_shift_2_deformer))

# Setup Dynamic Range compression
dynamic_range_compression_name = "drc"
dynamic_range_compression_deformer = muda.deformers.DynamicRangeCompression(preset=["music standard", "film standard", "speech", "radio"])

deformers_with_name.append((dynamic_range_compression_name, dynamic_range_compression_deformer))

audio_files_paths = sorted(glob.glob(audio_files_glob))
for deformer_name, deformer in deformers_with_name:
    print(f"Processing files with deformer: {deformer_name} ⚙️")

    for audio_file_path in tqdm.tqdm(audio_files_paths):
        deformed_audio_folder_path = f"{jams_files_path}/{deformer_name}/audio"
        deformed_jams_folder_path = f"{jams_files_path}/{deformer_name}/jams"
        os.makedirs(deformed_audio_folder_path, exist_ok=True)
        os.makedirs(deformed_jams_folder_path, exist_ok=True)
        file_name = os.path.basename(audio_file_path)[:-4]
        original_jam = muda.load_jam_audio(f"{original_jams_file_path}/{file_name}.jams", audio_file=audio_file_path)
        for i, deformed_jam in enumerate(deformer.transform(original_jam)):
            muda.save(f"{deformed_audio_folder_path}/{file_name}_{deformer_name}{i}.wav", f"{deformed_jams_folder_path}/{file_name}_{deformer_name}{i}.jams", deformed_jam)

    print(f"Done processing files with deformer: {deformer_name} ✅")

Processing files with deformer: bgnoise ⚙️


  0%|          | 0/888 [00:00<?, ?it/s]/Users/leol/Github/NAML-Urban-Sound-Classification/.venv/lib/python3.10/site-packages/muda/deformers/background.py:107: FutureWarning: Pass orig_sr=44100, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y = librosa.resample(y, soundf.samplerate, sr)
/Users/leol/Github/NAML-Urban-Sound-Classification/.venv/lib/python3.10/site-packages/muda/deformers/background.py:110: FutureWarning: Pass size=88200 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y = librosa.util.fix_length(y, n_samples)
/Users/leol/Github/NAML-Urban-Sound-Classification/.venv/lib/python3.10/site-packages/muda/deformers/background.py:107: FutureWarning: Pass orig_sr=44100, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y = librosa.resample(y, soundf.samplerate, sr)
  0%|          | 4/888 [00:03<10:01

In [24]:
# Create validation dataset from augmented dataset
from random import Random
import os
import glob
import tqdm

validation_split = 0.2
rng = Random(42)
fold = "fold1"
validation_dataset = f"data/UrbanSound8K/augmented_data/validation/{fold}"
train_dataset = f"data/UrbanSound8K/augmented_data/train/{fold}"

sample_names = list(map(lambda sample_path: sample_path.split("/")[-1][:-5], glob.glob(f"{train_dataset}/original/jams/*.jams")))
num_validation_samples = int(len(sample_names) * validation_split)

for i in tqdm.tqdm(range(num_validation_samples)):
    rand_sample_index = rng.randint(0, len(sample_names) - 1)
    sample_name_to_move = sample_names.pop(rand_sample_index)

    for modification_name in filter(lambda mod: not mod.startswith("."), os.listdir(train_dataset)):
        # Each modification may have created multiple copies of the original
        jams_file_glob = f"{train_dataset}/{modification_name}/jams/{sample_name_to_move}*.jams"
        for jams_file_path in glob.glob(jams_file_glob):
            os.rename(jams_file_path, jams_file_path.replace("train", "validation")) # move jams file
        audio_file_glob = f"{train_dataset}/{modification_name}/audio/{sample_name_to_move}*.wav"
        for audio_file_path in glob.glob(audio_file_glob):
            os.rename(audio_file_path, audio_file_path.replace("train", "validation")) # move wav file

  2%|▏         | 3/174 [00:00<00:07, 22.71it/s]

data/UrbanSound8K/augmented_data/train/fold1/drc/jams/180937-7-3-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/180937-7-3-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/180937-7-3-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/180937-7-3-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/180937-7-3-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/180937-7-3-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/122690-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/122690-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/122690-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/122690-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/122690-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/122690-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/97317-2-0-34*.jams
data/Ur

  5%|▌         | 9/174 [00:00<00:06, 24.94it/s]

data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/157867-8-0-3*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/157867-8-0-3*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/177621-0-0-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/177621-0-0-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/177621-0-0-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/177621-0-0-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/177621-0-0-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/177621-0-0-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/97317-2-0-23*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/97317-2-0-23*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/97317-2-0-23*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/97317-2-0-23*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/97317-2-0-23*.jams
data/Urban

  9%|▊         | 15/174 [00:00<00:06, 26.50it/s]

...

data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/161702-9-0-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/161702-9-0-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/161702-9-0-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/161702-9-0-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/161702-9-0-30*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/77246-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/77246-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/77246-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/77246-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/77246-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/77246-6-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/155310-3-0-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/155310-3-0-0*.jams
data/UrbanS

 95%|█████████▍| 165/174 [00:05<00:00, 29.63it/s]

data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/180937-7-2-2*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/174276-7-6-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/174276-7-6-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/174276-7-6-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/174276-7-6-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/174276-7-6-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/174276-7-6-0*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/78360-4-0-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/78360-4-0-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/78360-4-0-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/78360-4-0-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/78360-4-0-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/78360-4-0-1*.jams
data/UrbanSound

100%|██████████| 174/174 [00:06<00:00, 28.33it/s]

data/UrbanSound8K/augmented_data/train/fold1/drc/jams/143651-2-0-22*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/143651-2-0-22*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/143651-2-0-22*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/143651-2-0-22*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/143651-2-0-22*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/143651-2-0-22*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/103074-7-1-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams/103074-7-1-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/original/jams/103074-7-1-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/stretch/jams/103074-7-1-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/pitch1/jams/103074-7-1-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/bgnoise/jams/103074-7-1-1*.jams
data/UrbanSound8K/augmented_data/train/fold1/drc/jams/180937-7-1-14*.jams
data/U

In [17]:
# Generate command to undo the operations
validation_dataset = f"data/UrbanSound8K/augmented_data/validation/{fold}"
train_dataset = f"data/UrbanSound8K/augmented_data/train/{fold}"

for root, subdirs, files in os.walk(validation_dataset):
    if len(files) > 0:
        print(f"mv {root}/* {root.replace('validation', 'train')}")


mv data/UrbanSound8K/augmented_data/validation/fold1/drc/jams/* data/UrbanSound8K/augmented_data/train/fold1/drc/jams
mv data/UrbanSound8K/augmented_data/validation/fold1/drc/audio/* data/UrbanSound8K/augmented_data/train/fold1/drc/audio
mv data/UrbanSound8K/augmented_data/validation/fold1/pitch2/jams/* data/UrbanSound8K/augmented_data/train/fold1/pitch2/jams
mv data/UrbanSound8K/augmented_data/validation/fold1/pitch2/audio/* data/UrbanSound8K/augmented_data/train/fold1/pitch2/audio
mv data/UrbanSound8K/augmented_data/validation/fold1/original/jams/* data/UrbanSound8K/augmented_data/train/fold1/original/jams
mv data/UrbanSound8K/augmented_data/validation/fold1/original/audio/* data/UrbanSound8K/augmented_data/train/fold1/original/audio
mv data/UrbanSound8K/augmented_data/validation/fold1/stretch/jams/* data/UrbanSound8K/augmented_data/train/fold1/stretch/jams
mv data/UrbanSound8K/augmented_data/validation/fold1/stretch/audio/* data/UrbanSound8K/augmented_data/train/fold1/stretch/audio
